# XGBOOST feature selection
* check features from processed data
* train a XGB model with kCV where k = 10

In [3]:
# import libraries
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# libraries for feature selection
import os 
import glob

In [4]:
# terminal libraries, logger
from rich.logging import RichHandler
from rich.progress import track
import logging

FORMAT = "%(message)s"
logging.basicConfig(
    level="NOTSET", format=FORMAT, datefmt="[%X]", handlers=[RichHandler()]
)

log = logging.getLogger("rich")
log.info("Starting the program ...")


[18:39:36] INFO     Starting the program ...                                                       ]8;id=624799;file:///var/folders/5p/s9095ksd0zqfxtsgcx027bbr0000gn/T/ipykernel_96767/4041520490.py\4041520490.py]8;;\:]8;id=86227;file:///var/folders/5p/s9095ksd0zqfxtsgcx027bbr0000gn/T/ipykernel_96767/4041520490.py#12\12]8;;\

### functions

In [5]:
# Load the data
def load_and_combine_data(path):
    # Load all the csv files from the path
    os.chdir(path)
    extension = 'csv'
    all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    log.info(f"Loaded {len(all_filenames)} files from {path}.")

    def load_data(filepath):
        df = pd.read_csv(filepath)
        log.info(f"Loaded {filepath}.")
        return df

    combined_csv = pd.concat([load_data(f) for f in track(all_filenames, description="Loading CSV files...")])
    log.info(f"Combined {len(all_filenames)} files into one dataframe.")
    
    return combined_csv

# Load the data forma single csv file
def load_single_csv(path):
    df = pd.read_csv(path)
    log.info(f"Loaded {path}.")
    return df


In [6]:
from sklearn.model_selection import KFold

# function to train XGBoost model
def train_xgboost(X_train, X_test, y_train, y_test):
    model = GradientBoostingClassifier().fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return y_pred, model


# function to train Random Forest model
def train_random_forest(X_train, X_test, y_train, y_test):
    model = RandomForestClassifier().fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return y_pred, model

In [7]:
# function to plot confusion matrix
def plot_confusion_matrix(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='g')
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

# function to plot accuracy
def plot_accuracy(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    plt.figure(figsize=(10, 7))
    sns.barplot(x=['Accuracy'], y=[accuracy])
    plt.title('Accuracy')
    plt.show()

# function to plot mean squared error
def plot_mse(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    plt.figure(figsize=(10, 7))
    sns.barplot(x=['MSE'], y=[mse])
    plt.title('Mean Squared Error')
    plt.show()

# function to plot actual vs predicted
def plot_actual_vs_predicted(y_test, y_pred):
    plt.figure(figsize=(10, 7))
    plt.plot(y_test, label='Actual')
    plt.plot(y_pred, label='Predicted')
    plt.legend()
    plt.title('Actual vs Predicted')
    plt.show()

# function to plot feature importance
def plot_feature_importance(model, features):
    plt.figure(figsize=(10, 7))
    sns.barplot(x=model.feature_importances_, y=features.columns)
    plt.title('Feature Importance')
    plt.show()


In [8]:
def k_fold_xg(df, k=10):
    # Split the data into features and target
    X = df.drop(['grasp', 'ts'], axis=1)
    y = df['grasp']

    # Define the k-fold cross-validation
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    # Initialize variables to keep track of the best model
    best_accuracy = 0
    best_model = None
    best_model_type = ''
    best_X_test = None
    best_y_test = None

    # Loop through all folds
    for fold, (train_index, test_index) in track(enumerate(kf.split(X, y)), description="Running k-fold cross-validation..."):
        log.info(f"Starting fold {fold + 1}/{k}...")

        # Split data into train and test sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        log.info(f"Split data into train and test sets for fold {fold + 1}/{k}.")

        # Train and evaluate XGBoost model
        y_pred_xgb, xg_model = train_xgboost(X_train, X_test, y_train, y_test)
        accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
        log.info(f"Trained XGBoost model for fold {fold + 1}/{k}.")

        # Train and evaluate Random Forest model
        y_pred_rf, rf_model = train_random_forest(X_train, X_test, y_train, y_test)
        accuracy_rf = accuracy_score(y_test, y_pred_rf)
        log.info(f"Trained Random Forest model for fold {fold + 1}/{k}.")

        # Check which model performed better and save it if it's the best so far
        if accuracy_xgb > accuracy_rf and accuracy_xgb > best_accuracy:
            best_accuracy = accuracy_xgb
            best_model = xg_model
            best_model_type = 'XGBoost'
            best_X_test = X_test
            best_y_test = y_test
            log.info(f"Best model is XGBoost with accuracy {best_accuracy}. In fold {fold + 1}/{k}.")
        elif accuracy_rf > accuracy_xgb and accuracy_rf > best_accuracy:
            best_accuracy = accuracy_rf
            best_model = rf_model
            best_model_type = 'Random Forest'
            best_X_test = X_test
            best_y_test = y_test
            log.info(f"Best model is Random Forest with accuracy {best_accuracy}. In fold {fold + 1}/{k}.")

        log.info(f"Completed fold {fold + 1}/{k}.")

    # After all folds, plot the best model's results
    log.info(f"Best model is {best_model_type} with accuracy {best_accuracy}.")
    best_y_pred = best_model.predict(best_X_test)
    plot_confusion_matrix(best_y_test, best_y_pred)
    plot_accuracy(best_y_test, best_y_pred)
    plot_mse(best_y_test, best_y_pred)
    plot_actual_vs_predicted(best_y_test, best_y_pred)
    plot_feature_importance(best_model, X.dropna(axis=1))

    log.info(f"Finished k-fold cross-validation.")

## MAIN

In [13]:
# ---------------------------------------------------------------------------------------------
# MAIN:

# PATHS
path = "data/processed_data/"
path_single = "../data/processed_data/subject_010.csv"

# Load the data
df = load_single_csv(path_single)
log.info(df.head())

# Print only the grasp and ts columns
log.info(df[['grasp', 'ts']].head())

[18:43:05] INFO     Loaded ../data/processed_data/subject_010.csv.                                 ]8;id=772804;file:///var/folders/5p/s9095ksd0zqfxtsgcx027bbr0000gn/T/ipykernel_96767/1604078616.py\1604078616.py]8;;\:]8;id=561580;file:///var/folders/5p/s9095ksd0zqfxtsgcx027bbr0000gn/T/ipykernel_96767/1604078616.py#22\22]8;;\

           INFO            MAV0       ZC0      SSC0       WL0      RMS0          VAR0      LOG0  \ ]8;id=791234;file:///var/folders/5p/s9095ksd0zqfxtsgcx027bbr0000gn/T/ipykernel_96767/1951632961.py\1951632961.py]8;;\:]8;id=195693;file:///var/folders/5p/s9095ksd0zqfxtsgcx027bbr0000gn/T/ipykernel_96767/1951632961.py#10\10]8;;\
                    0  0.000003  0.156250  0.338542  0.000001  0.000004  1.612673e-11  0.000002                    
                    1  0.000003  0.202073  0.336788  0.000002  0.000005  2.599275e-11  0.000002                    
                    2  0.000003  0.145078  0.331606  0.000002  0.000005  2.454870e-11  0.000002                    
                    3  0.000003  0.171875  0.322917  0.000001  0.000004  1.272511e-11  0.000002                    
                    4  0.000002  0.176166  0.321244  0.000001  0.000003  1.033333e-11  0.000002                    
                                                                                                                   
                          IEMG0  MNF_real0  MNF_imag0  ...  SKF_real11  SKF_imag11  KUF_real11  \                  
                    0  0.000602   0.107012   0.104630  ...    0.250060    0.250043    0.250080                     
                    1  0.000648   0.103550   0.109092  ...    0.248748    0.248751    0.248763                     
                    2  0.000669   0.101020   0.106314  ...    0.248784    0.248748    0.248810                     
                    3  0.000516   0.127651   0.112898  ...    0.250032    0.250047    0.250043                     
                    4  0.000475   0.126625   0.117621  ...    0.248745    0.248766    0.248759                     
                                                                                                                   
                       KUF_imag11  MAXPSD11_real  MAXPSD11_imag  MINPSD11_real  MINPSD11_imag  \                   
                    0    0.250058   2.136455e-08   1.394703e-08   1.196153e-15            0.0                      
                    1    0.248766   2.194821e-08   8.373398e-09   8.587776e-14            0.0                      
                    2    0.248762   2.696580e-08   3.758184e-08   4.780079e-16            0.0                      
                    3    0.250062   1.271739e-08   1.203147e-08   2.257072e-14            0.0                      
                    4    0.248786   1.748059e-08   2.126701e-08   8.555701e-14            0.0                      
                                                                                                                   
                               ts  grasp                                                                           
                    0  173.424297      1                                                                           
                    1  173.524246      1                                                                           
                    2  173.623936      1                                                                           
                    3  173.723885      1                                                                           
                    4  173.823834      1                                                                           
                                                                                                                   
                    [5 rows x 350 columns]                                                                         

           INFO        grasp          ts                                                           ]8;id=57513;file:///var/folders/5p/s9095ksd0zqfxtsgcx027bbr0000gn/T/ipykernel_96767/1951632961.py\1951632961.py]8;;\:]8;id=792467;file:///var/folders/5p/s9095ksd0zqfxtsgcx027bbr0000gn/T/ipykernel_96767/1951632961.py#13\13]8;;\
                    0      1  173.424297                                                                           
                    1      1  173.524246                                                                           
                    2      1  173.623936                                                                           
                    3      1  173.723885                                                                           
                    4      1  173.823834                                                                           

In [14]:
# To run k-fold cross-validation
k_fold_xg(df, k=10)

Output()

[18:43:17] INFO     Starting fold 1/10...                                                          ]8;id=933405;file:///var/folders/5p/s9095ksd0zqfxtsgcx027bbr0000gn/T/ipykernel_96767/4074503103.py\4074503103.py]8;;\:]8;id=44975;file:///var/folders/5p/s9095ksd0zqfxtsgcx027bbr0000gn/T/ipykernel_96767/4074503103.py#18\18]8;;\

           INFO     Split data into train and test sets for fold 1/10.                             ]8;id=989507;file:///var/folders/5p/s9095ksd0zqfxtsgcx027bbr0000gn/T/ipykernel_96767/4074503103.py\4074503103.py]8;;\:]8;id=212309;file:///var/folders/5p/s9095ksd0zqfxtsgcx027bbr0000gn/T/ipykernel_96767/4074503103.py#23\23]8;;\

KeyboardInterrupt: 

           DEBUG    handle_msg[1352b544531545f6854c4de845278dcf]({'header': {'date':               ]8;id=640138;file:///Users/albertoquintana/miniconda3/envs/ml/lib/python3.11/site-packages/comm/base_comm.py\base_comm.py]8;;\:]8;id=865221;file:///Users/albertoquintana/miniconda3/envs/ml/lib/python3.11/site-packages/comm/base_comm.py#150\150]8;;\
                    datetime.datetime(2023, 10, 9, 16, 43, 28, 622000, tzinfo=tzutc()), 'msg_id':                  
                    '204a29a1-3e39-4e12-8c35-b9fecb824fc2', 'msg_type': 'comm_msg', 'session':                     
                    'ac00f2ef-d869-4843-a1cb-cd1e22c165ae', 'username':                                            
                    '7f582236-5886-404f-a2bd-a86a3955d991', 'version': '5.2'}, 'msg_id':                           
                    '204a29a1-3e39-4e12-8c35-b9fecb824fc2', 'msg_type': 'comm_msg',                                
                    'parent_header': {}, 'metadata': {}, 'content': {'comm_id':                                    
                    '1352b544531545f6854c4de845278dcf', 'data': {'method': 'update', 'state':                      
                    {'outputs': [{'output_type': 'display_data', 'data': {'text/plain': 'Running                   
                    k-fold cross-validation...                                                                     
                    \x1b[38;2;183;44;94m━\x1b[0m\x1b[38;2;153;48;86m━\x1b[0m\x1b[38;2;123;51;77m━\                 
                    x1b[0m\x1b[38;2;97;53;69m━\x1b[0m\x1b[38;2;76;56;63m━\x1b[0m\x1b[38;2;62;57;59                 
                    m━\x1b[0m\x1b[38;2;58;58;58m━\x1b[0m\x1b[38;2;62;57;59m━\x1b[0m\x1b[38;2;76;56                 
                    ;63m━\x1b[0m\x1b[38;2;97;53;69m━\x1b[0m\x1b[38;2;123;51;77m━\x1b[0m\x1b[38;2;1                 
                    53;48;86m━\x1b[0m\x1b[38;2;183;44;94m━\x1b[0m\x1b[38;2;209;42;102m━\x1b[0m\x1b                 
                    [38;2;230;39;108m━\x1b[0m\x1b[38;2;244;38;112m━\x1b[0m\x1b[38;2;249;38;114m━\x                 
                    1b[0m\x1b[38;2;244;38;112m━\x1b[0m\x1b[38;2;230;39;108m━\x1b[0m\x1b[38;2;209;4                 
                    2;102m━\x1b[0m\x1b[38;2;183;44;94m━\x1b[0m\x1b[38;2;153;48;86m━\x1b[0m\x1b[38;                 
                    2;123;51;77m━\x1b[0m\x1b[38;2;97;53;69m━\x1b[0m\x1b[38;2;76;56;63m━\x1b[0m\x1b                 
                    [38;2;62;57;59m━\x1b[0m\x1b[38;2;58;58;58m━\x1b[0m\x1b[38;2;62;57;59m━\x1b[0m\                 
                    x1b[38;2;76;56;63m━\x1b[0m\x1b[38;2;97;53;69m━\x1b[0m\x1b[38;2;123;51;77m━\x1b                 
                    [0m\x1b[38;2;153;48;86m━\x1b[0m\x1b[38;2;183;44;94m━\x1b[0m\x1b[38;2;209;42;10                 
                    2m━\x1b[0m\x1b[38;2;230;39;108m━\x1b[0m\x1b[38;2;244;38;112m━\x1b[0m\x1b[38;2;                 
                    249;38;114m━\x1b[0m\x1b[38;2;244;38;112m━\x1b[0m\x1b[38;2;230;39;108m━\x1b[0m\                 
                    x1b[38;2;209;42;102m━\x1b[0m  \x1b[36m \x1b[0m\n', 'text/html': '<pre                          
                    style="white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,\'                 
                    DejaVu Sans Mono\',consolas,\'Courier New\',monospace">Running k-fold                          
                    cross-validation... <span style="color: #b72c5e; text-decoration-color:                        
                    #b72c5e">━</span><span style="color: #993056; text-decoration-color:                           
                    #993056">━</span><span style="color: #7b334d; text-decoration-color:                           
                    #7b334d">━</span><span style="color: #613545; text-decoration-color:                           
                    #613545">━</span><span style="color: #4c383f; text-decoration-color:                           
                    #4c383f">━</span><span style